In [2]:
import numpy as np

import ModelDefinition

import SupplyChainLocations
import SupplyChainRules
import json

In [3]:
# seeding_rate in tonnes/ha (hectacre is 2.47105 acres)
# 39.3679 bushels of wheat is a tonne 
model_constants = {
    "Wheat_seeding_rate":0.11,
    "Cereals_seeding_rate":0.2,
    "Barley_seeding_rate":0.3,
    "Wheat_germination_rate":1
}

In [4]:
supplyChainClasses = [["NH4", "tonnes"], ["N2", "m^3"], ["H2", "m^3"], ["CH4", "m^3"]]


# Crop classes:
crops = ["Wheat", "Cereals", "Barley"]

for crop in crops:
    supplyChainClasses +=  [[f"Seeds_{crop}", "Tonnes"], [f"Planted_{crop}", "Hectacres"], [f"Growing_{crop}", "Hectacres"], [f"Viable_{crop}", "Tonnes"], [f"Harvested_{crop}", "Tonnes"]]

In [5]:
def returnCropRules():    
    rules = []

    # Crop rules
    # TODO add seasonallity.
    # TODO limit based on supply.
    for crop in crops:
        # SIMPLIFIED RULE FOR BASE MODEL
        # Fix requirement of non None propensity class.
        purchase_seeds = SupplyChainRules.ExitEntranceRule(target="FarmRegion", transport_class=f"Seeds_{crop}", transport_amount=1, propensity="0.1", propensity_classes=[f"Seeds_{crop}"],
                                                           rule_name="Purchase Seeds")

        sow_crop = SupplyChainRules.SingleLocationProductionRule(target="FarmRegion",
                                                                        reactant_classes=[f"Seeds_{crop}"], reactant_amount=[1], 
                                                                        product_classes=[f"Planted_{crop}"], product_amount=[model_constants[f"{crop}_seeding_rate"]], propensity=f"Seeds_{crop}",
                                                                        propensity_classes=[f"Seeds_{crop}"], rule_name=f"Sow {crop}")
        
        germinating_crop = SupplyChainRules.SingleLocationProductionRule(target="FarmRegion",
                                                                        reactant_classes=[f"Planted_{crop}"], reactant_amount=[1], 
                                                                        product_classes=[f"Growing_{crop}"], product_amount=[1], propensity=f"0.1*Planted_{crop}",
                                                                        propensity_classes=[f"Planted_{crop}"], rule_name=f"Germinate {crop}")
        # TODO make dependant on Nitrogen levels ect ect.
        crop_growth = SupplyChainRules.SingleLocationProductionRule(target="FarmRegion",
                                                                        reactant_classes=[f"Growing_{crop}"], reactant_amount=[1], 
                                                                        product_classes=[f"Viable_{crop}"], product_amount=[1], propensity=f"Growing_{crop}",
                                                                        propensity_classes=[f"Growing_{crop}"], rule_name=f"Germinate {crop}")
        
        # May need to make propensity based on planted field size, will need to check.
        harvest_crop = SupplyChainRules.SingleLocationProductionRule(target="FarmRegion",
                                                                        reactant_classes=[f"Viable_{crop}"], reactant_amount=[1], 
                                                                        product_classes=[f"Harvested_{crop}"], product_amount=[1], propensity=f"Viable_{crop}",
                                                                        propensity_classes=[f"Viable_{crop}"], rule_name=f"Harvest {crop}")
        rules += [purchase_seeds, sow_crop, germinating_crop, crop_growth, harvest_crop]

    return rules

In [6]:
def returnFertiliserRules():
    return []

In [7]:
def supplyChainRules():
    rules = []
    rules += returnCropRules()
    rules += returnFertiliserRules()
    return rules

In [8]:
def supplyChainLocations():
    # Billingham terminal - Produces Ammonium nitrate from Ammonia
    # https://www.cfindustries.com/newsroom/2023/billingham-ammonia-plant
    # https://www.cfindustries.com/what-we-do/fertilizer
    all_locations = []
    # Midpoints from wikipedia, South East and London uses South East midpoint (combined to match DEFRA reporting)
    region_infos = [[54.075, -2.75, "North East"], [55, -1.87, "North West"], [53.566667, -1.2, "Yorkshire & The Humber"], [52.98, -0.75, "East Midlands"], [52.478861, -2.256306, "West Midlands"], 
                    [52.24, 0.41, "East of England"], [51.3, -0.8, "South East & London"], [50.96, -3.22, "South West"], [56.816738, -4.183963, "Scotland"], [52.33022, -3.766409,"Wales"]]
    for region_info in region_infos:
        all_locations.append(SupplyChainLocations.FarmRegion(crops, *region_info))

    
    return all_locations

In [9]:
model = ModelDefinition.ModelDefinition(supplyChainClasses, supplyChainLocations, supplyChainRules, model_folder="../ModelFiles/SupplyChain/")
model.build()

0.1
0.100000000000000
Seeds_Wheat
1.00000000000000
0.1*Planted_Wheat
0.100000000000000
Growing_Wheat
1.00000000000000
Viable_Wheat
1.00000000000000
0.1
0.100000000000000
Seeds_Cereals
1.00000000000000
0.1*Planted_Cereals
0.100000000000000
Growing_Cereals
1.00000000000000
Viable_Cereals
1.00000000000000
0.1
0.100000000000000
Seeds_Barley
1.00000000000000
0.1*Planted_Barley
0.100000000000000
Growing_Barley
1.00000000000000
Viable_Barley
1.00000000000000
['Seeds_Wheat']
['Seeds_Wheat', 'Planted_Wheat']
['Planted_Wheat', 'Growing_Wheat']
['Growing_Wheat', 'Viable_Wheat']
['Viable_Wheat', 'Harvested_Wheat']
['Seeds_Cereals']
['Seeds_Cereals', 'Planted_Cereals']
['Planted_Cereals', 'Growing_Cereals']
['Growing_Cereals', 'Viable_Cereals']
['Viable_Cereals', 'Harvested_Cereals']
['Seeds_Barley']
['Seeds_Barley', 'Planted_Barley']
['Planted_Barley', 'Growing_Barley']
['Growing_Barley', 'Viable_Barley']
['Viable_Barley', 'Harvested_Barley']
MATCHED {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
MATCHED {0: